In [1]:
import sympy
import sys
import numpy as np
sys.path.append('F:\\MML_Oller\\tensorconvert\\src\\')
from tensorconvert import SecondOrderTensor, FourthOrderTensor

sympy.init_printing(use_latex="mathjax")

In [2]:
def linear_elastic(eps):
    dim = eps.shape[0]
    lmbda, mu = sympy.symbols("lambda, mu", positive=True)
    f = lmbda * sympy.trace(eps) * sympy.eye(dim) + 2 * mu * eps
    f = f.subs({lmbda: 7.479e6, mu: 3.751e6})
    return f

In [3]:
C4_obj = FourthOrderTensor().from_operator(linear_elastic)
C4_sp = C4_obj.as_array()

In [4]:
C4 = np.zeros((3,3,3,3))
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                C4[i,j,k,l] = C4_sp[i,j,k,l]
            


In [5]:
C4_obj.as_mandel()

⎡14981000.0  7479000.0   7479000.0       0          0          0    ⎤
⎢                                                                   ⎥
⎢7479000.0   14981000.0  7479000.0       0          0          0    ⎥
⎢                                                                   ⎥
⎢7479000.0   7479000.0   14981000.0      0          0          0    ⎥
⎢                                                                   ⎥
⎢    0           0           0       7502000.0      0          0    ⎥
⎢                                                                   ⎥
⎢    0           0           0           0      7502000.0      0    ⎥
⎢                                                                   ⎥
⎣    0           0           0           0          0      7502000.0⎦

In [6]:
#sv = np.random.rand(6)*1000
sv =  [   0.,            0.,            0.,           0. ,           0., 1382.32346212]
stress_tensor = np.array([[sv[0], sv[5], sv[4]], [ sv[5], sv[1], sv[3] ], [ sv[4], sv[3], sv[2] ] ])
stress_tensor

array([[   0.        , 1382.32346212,    0.        ],
       [1382.32346212,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ]])

In [7]:
stress_obj = SecondOrderTensor().from_array(stress_tensor)
stress_vec = stress_obj.as_voigt_stress()
stress_vec

⎡      0      ⎤
⎢             ⎥
⎢      0      ⎥
⎢             ⎥
⎢      0      ⎥
⎢             ⎥
⎢1382.32346212⎥
⎢             ⎥
⎢      0      ⎥
⎢             ⎥
⎣      0      ⎦

In [8]:
import math
ROOT2 = math.sqrt(2)
def dGdSVoigt(stress_vector):
    sigma_1, sigma_2, sigma_3, sigma_4, sigma_5, sigma_6 = stress_vector
    sigma_4 = sigma_4
    sigma_5 = sigma_5
    sigma_6 = sigma_6
    denom = math.sqrt(6*sigma_4**2 + 6*sigma_5**2 + 6*sigma_6**2 + (sigma_1 - sigma_2)**2 + (sigma_1 - sigma_3)**2 + (sigma_2 - sigma_3)**2)
    dGds1 = ROOT2*(2*sigma_1 - sigma_2 - sigma_3)/(2*denom)
    dGds2 = ROOT2*(-sigma_1 + 2*sigma_2 - sigma_3)/(2*denom)
    dGds3 = ROOT2*(-sigma_1 - sigma_2 + 2*sigma_3)/(2*denom)
    dGds4 = 3*ROOT2*sigma_4/denom
    dGds5 = 3*ROOT2*sigma_5/denom
    dGds6 = 3*ROOT2*sigma_6/denom
    dGdS_array = np.array([dGds1, dGds2, dGds3, dGds4, dGds5, dGds6])
    #dGdS_array[3:] = dGdS_array[3:]
    return dGdS_array
    
dYv = dGdSVoigt(stress_vec)
dGdS_tensor = np.array([[dYv[0], dYv[3], dYv[4]], [dYv[3],dYv[1],dYv[5]], [dYv[4],dYv[5],dYv[2]]])
print(dYv, '\n', dGdS_tensor)

[0 0 0 1.73205080756888 0 0] 
 [[0 1.73205080756888 0]
 [1.73205080756888 0 0]
 [0 0 0]]


In [9]:
np.einsum('ij,ij', dGdS_tensor, np.einsum('ijkl,kl', C4, dGdS_tensor))

45012000.0000000

In [10]:
dGdS_obj = SecondOrderTensor().from_array(dGdS_tensor)
dGdS_obj.as_mandel()

⎡         0         ⎤
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢1.73205080756888⋅√2⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎣         0         ⎦

In [11]:
dYv_copy = dYv.copy()
dYv_copy[3:] = dYv_copy[3:] * ROOT2
dYv_copy @ np.array(C4_obj.as_mandel()) @ dYv_copy

45012000.0000000

In [12]:
def vm_stress_mandell(mandel_stress_vec):
    Y = 1e3
    # MML stress comes in the following order: 11, 22, 33, 12, 23, 13
    sigma_11, sigma_22, sigma_33, sigma_12, sigma_23, sigma_13 = mandel_stress_vec
    sigma_12 = sigma_12/ROOT2
    sigma_23 = sigma_23/ROOT2
    sigma_13 = sigma_13/ROOT2
    internal = (sigma_11 - sigma_22)**2 + (sigma_22 - sigma_33)**2 + (sigma_33 - sigma_11)**2 + 6*(sigma_23**2 + sigma_13**2 + sigma_12**2)
    vm = math.sqrt(0.5*internal)
    return vm - Y
vm_stress_mandell(np.array(stress_obj.as_mandel()))

C:\Users\brian\AppData\Local\Temp\ipykernel_65428\78138145.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  vm = math.sqrt(0.5*internal)


1394.2544688863518